# **distilbert-base-uncased基于PyTorch做微调训练**

# 1.安装环境

In [ ]:
%%writefile requirements.txt
datasets
transformers
evaluate
torch
torchvision
torchaudio

In [ ]:
pip install -r requirements.txt

# 2.加载数据集
## 数据集已经提前下载到本地

In [ ]:
from datasets import load_dataset

# 远程拉取
# dataset = load_dataset("yelp_review_full")

dataset = load_dataset(
    'parquet',
    data_files={
        'train': '../datasets/yelp_review_full/train-00000-of-00001.parquet',
        'test': '../datasets/yelp_review_full/test-00000-of-00001.parquet'
    }
)

dataset["train"][100]

## 文本分词

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("../models/distilbert-base-uncased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

## 数据处理

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

## 数据筛选

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

## 加载数据

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=4)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=4)

# 3.准备训练

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("../models/distilbert-base-uncased", num_labels=5)

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)
model.to(device)

# 4.开始训练

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

# 5.训练评估

In [ ]:
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

# 6.清理环境

In [ ]:
del model
del tokenizer
del dataset
del tokenized_datasets
del small_train_dataset
del small_eval_dataset
del train_dataloader
del eval_dataloader

In [ ]:
pip uninstall -y -r requirements.txt

In [ ]:
rm -rf requirements.txt